Заполните данные о необходимых переменных:

In [1]:
RUB_USD = 0.010309087
EUR_USD = 1.052536139

Today = '31.12.2023'

# Group = 'SUEK'
Group = 'EuroChem'

Data_file = 'Credit_portfolio_SAP' + '.xlsx'
Sheet_in_data_file = 'Export Worksheet'

Mapping_file = 'C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\Методология\\Mapping.xlsx'
Sheet_in_mapping_file = 'mapping'

from datetime import date
Output_file = "_".join([str(date.today()), Group, 'SAP', 'Debt.xlsx'])
Sheet_in_output_file = 'Debt'

Открываю следующие файлы:

In [2]:
import pandas as pd
import numpy as np

data_export = pd.read_excel(Data_file, sheet_name = Sheet_in_data_file)
data_export[['businessPartnerName', 'companyName']] = data_export[['businessPartnerName', 'companyName']].apply(lambda x: x.str.lower())

In [3]:
data_mapping = pd.read_excel(Mapping_file, sheet_name = Sheet_in_mapping_file)
data_mapping.Code = data_mapping.Code.str.lower()

In [9]:
data_export = data_export.query('Group == @Group').reset_index(drop=True)
data_Debt = data_export.loc[data_export['productTypeName'].isin(['Транш по займам', 'Транш по кредитам'])]

data_Debt = data_Debt.groupby(['companyName', 'relationshipPartner',
       'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
                                                                          'purchaseValuePositionCurrency' : sum}).reset_index()

data_Debt['in_USD'] = data_Debt['purchaseValuePositionCurrency'] * np.sign(data_Debt['purchaseValueDisplayCurrency'])
data_Debt.loc[data_Debt['creditConditionCurrency'] == 'RUB', 'in_USD'] = data_Debt.loc[data_Debt['creditConditionCurrency'] == 'RUB', 'in_USD'] * RUB_USD
data_Debt.loc[data_Debt['creditConditionCurrency'] == 'EUR', 'in_USD'] = data_Debt.loc[data_Debt['creditConditionCurrency'] == 'EUR', 'in_USD'] * EUR_USD
data_Debt['purchaseValuePositionCurrency'] = data_Debt['purchaseValuePositionCurrency'] * np.sign(data_Debt['purchaseValueDisplayCurrency'])

data_Debt = data_Debt.rename(columns={"businessPartnerName": "Entity", "companyName": "Counterparty", "creditConditionCurrency": "Currency", "purchaseValuePositionCurrency": "amoutn_outstanding", "in_USD": "amount_USD_eq"})

data_Debt['Entity_group'] = data_Debt\
                                   .merge(data_mapping, how = 'left', left_on = 'Entity', right_on = 'Code')['Business_segment']\
                                   .replace({np.NaN : 'External'})\
                                   .reset_index(drop=True)
data_Debt['Counterparty_Group'] = data_Debt\
                                   .merge(data_mapping, how = 'left', left_on = 'Counterparty', right_on = 'Code')['Business_segment']\
                                   .replace({np.NaN : 'External'})\
                                   .reset_index(drop=True)

day = pd.to_datetime(Today, dayfirst = True)
data_Debt['Days'] = 0
data_Debt.termEnd = pd.to_datetime(data_Debt['termEnd'], errors='coerce')
data_Debt['Days'] = data_Debt.loc[data_Debt['termEnd'] != np.NaN, 'termEnd'] - day
data_Debt['Days'] = data_Debt['Days'].dt.days
data_Debt['Period'] = '2Y+'
data_Debt.loc[pd.to_numeric(data_Debt['Days']) < 730, 'Period'] = '1Y-2Y'
data_Debt.loc[pd.to_numeric(data_Debt['Days']) < 365, 'Period'] = '6M-1Y'
data_Debt.loc[pd.to_numeric(data_Debt['Days']) < 182, 'Period'] = '3M-6M'
data_Debt.loc[pd.to_numeric(data_Debt['Days']) < 91, 'Period'] = '1M-3M'
data_Debt.loc[pd.to_numeric(data_Debt['Days']) < 30, 'Period'] = '<1M'

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_19424\1171638031.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_19424\1171638031.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,


Записываю данные в output file:

In [10]:
new_columns = ['dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']
data_Debt = data_Debt.reindex(columns=(data_Debt.columns.tolist() + new_columns))
data_Debt['Source'] = 'SAP'

data = data_Debt[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
data[['Entity', 'Counterparty']] = data[['Entity', 'Counterparty']].apply(lambda x: x.str.upper())

data.to_excel(Output_file, sheet_name = Sheet_in_output_file, index = False)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_19424\4245928966.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Entity', 'Counterparty']] = data[['Entity', 'Counterparty']].apply(lambda x: x.str.upper())
